In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import randint, uniform

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

# -------------------------------
# 1️⃣ Création de caractéristiques avancées
# -------------------------------
def create_advanced_features(df):
    df['loan_per_employee'] = df['GrAppv'] / (df['NoEmp'] + 1)
    df['job_creation_rate'] = (df['CreateJob'] + df['RetainedJob']) / (df['NoEmp'] + 1)
    
    df['term_amount_ratio'] = df['GrAppv'] / df['Term'].replace(0, np.nan)
    
    state_default_rates = df.groupby('State')['MIS_Status'].mean()
    df['state_risk'] = df['State'].map(state_default_rates)
    
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df

# Appliquer la transformation
df = create_advanced_features(df)

# Séparation des features et de la target
X = df.drop(columns=['MIS_Status'])
y = df['MIS_Status']

# Définition des variables catégorielles
cat_features = ['State', 'NewExist', 'UrbanRural', 'FranchiseCode', 'NAICS', 'ApprovalFY','crysis_year']

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Convertir les colonnes catégorielles en string
X_train[cat_features] = X_train[cat_features].astype(str)
X_test[cat_features] = X_test[cat_features].astype(str)

# Vérifier si les colonnes catégorielles existent bien
for col in cat_features:
    if col not in X_train.columns:
        print(f"⚠️ Attention: {col} n'est pas présent dans X_train.")

# -------------------------------
# 2️⃣ Grid Search optimisé
# -------------------------------
param_grid = {
    'depth': [4, 5, 6],  
    'grow_policy': ['Lossguide'],  
    'l2_leaf_reg': [0.5, 0.6, 0.7],  
    'learning_rate': [0.16, 0.18, 0.20],  
    'scale_pos_weight': [1]  
}

grid_search = GridSearchCV(
    estimator=CatBoostClassifier(
        iterations=500,  
        loss_function='Logloss',
        cat_features=cat_features,
        early_stopping_rounds=50,  
        verbose=200  
    ),
    param_grid=param_grid,
    cv=3,  
    scoring='roc_auc',
    verbose=3,
    n_jobs=-1  
)

# Exécution du GridSearch
grid_search.fit(X_train, y_train)

# Meilleurs paramètres
print("Meilleurs paramètres après GridSearch :", grid_search.best_params_)

# -------------------------------
# 3️⃣ Réentraînement avec les meilleurs paramètres
# -------------------------------
best_model = CatBoostClassifier(
    **grid_search.best_params_,
    iterations=1000,  
    loss_function='Logloss',
    cat_features=cat_features,
    early_stopping_rounds=100,
    verbose=200
)

best_model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    plot=True
)

# -------------------------------
# 4️⃣ Prédictions avec seuil personnalisé (0.3)
# -------------------------------
# Probabilités de prédiction
y_pred_proba_train = best_model.predict_proba(X_train)[:, 1]
y_pred_proba_test = best_model.predict_proba(X_test)[:, 1]

# Définition du seuil
seuil = 0.3

# Conversion en classes en fonction du seuil
y_pred_train_custom = (y_pred_proba_train >= seuil).astype(int)
y_pred_test_custom = (y_pred_proba_test >= seuil).astype(int)

# Affichage des premières valeurs
print("📊 Exemple de prédictions avec seuil personnalisé :")
print("Train:", y_pred_train_custom[:10])
print("Test:", y_pred_test_custom[:10])
